In [2]:
#To create a csv file. This csv file contains:
#All 16 models' forecasting of precipitation for all dates, for all grids. 
#Where the forecasting for the 8th day is the forecast of the latest weekly forecast.
#Imports

In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [6]:
#Main
base_path = r'C:\Users\user\Downloads\erfs_force\media\adam\WCL_URMIN\StreamflowForecast\Forcing2003_2018'
output_csv = 'forecasted.csv'
all_data = []

In [8]:
for issue_folder in sorted(os.listdir(base_path)):
    issue_path = os.path.join(base_path, issue_folder)
    if not os.path.isdir(issue_path):
        continue

    issue_date = datetime.strptime(issue_folder, "%Y%m%d")
    if issue_date.month not in [6, 7, 8, 9]:
        continue

    print(f"Processing folder: {issue_folder}")

    for model_name in os.listdir(issue_path):
        model_path = os.path.join(issue_path, model_name)
        if not os.path.isdir(model_path):
            continue

        for filename in os.listdir(model_path):
            if not filename.startswith("data_"):
                continue

            lat, lon = map(float, filename.replace("data_", "").split("_"))
            file_path = os.path.join(model_path, filename)
            data = np.loadtxt(file_path, max_rows=7, usecols=(0,))

            for i, precip in enumerate(data):
                forecast_date = (issue_date + timedelta(days=i + 1)).strftime("%Y-%m-%d")
                all_data.append([forecast_date, model_name, lat, lon, precip])

In [9]:
df=pd.DataFrame(all_data, columns=["Forecast_Date", "Model", "Lat", "Lon", "Precipitation"])
df.to_csv(output_csv, index=False)

In [11]:
df=pd.read_csv("forecasted.csv")

In [14]:
df.head(2)

,Forecast_Date,Model,Lat,Lon,Precipitation
0,2013-08-02,CFST126_00,21.125,82.125,5.12190
1,2013-08-03,CFST126_00,21.125,82.125,1.45660
2,2013-08-04,CFST126_00,21.125,82.125,0.49005
3,2013-08-05,CFST126_00,21.125,82.125,0.78300
4,2013-08-06,CFST126_00,21.125,82.125,4.83570


In [1]:
observedf="observed"
observed_all = []

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for filename in os.listdir(observedf):
    if not filename.startswith("data_"):
        continue

    lat, lon = map(float, filename.replace("data_", "").split("_"))
    obs_filep = os.path.join(observedf, filename)

    obs_df = pd.read_csv(
        obs_filep,
        delim_whitespace=True,
        header=None,
        names=["Year", "Month", "Day", "Observed_Precip"]
    )
    obs_df["Date"] = pd.to_datetime(obs_df[["Year", "Month", "Day"]])
    obs_df = obs_df.drop(columns=["Year", "Month", "Day"])
    obs_df["Lat"] = lat
    obs_df["Lon"] = lon

    observed_all.append(obs_df)

In [ ]:
full_obs_df = pd.concat(observed_all)
full_obs_df.set_index("Date", inplace=True)
full_obs_df.sort_index(inplace=True)
full_obs_df.head(2)

In [ ]:
full_obs_df.to_csv('observeddata.csv')

In [2]:
#End